# TPU Configuration

In [3]:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl -qq

In [4]:
# import torch_xla
# import torch_xla.debug.metrics as met
# import torch_xla.distributed.parallel_loader as pl
# import torch_xla.utils.utils as xu
# import torch_xla.core.xla_model as xm
# import torch_xla.distributed.xla_multiprocessing as xmp
# import torch_xla.test.test_utils as test_utils

# import warnings
# warnings.filterwarnings("ignore")
# device = xm.xla_device()

# Library

In [5]:
! pip install Korpora sentencepiece einops wandb torch-summary -qq

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from Korpora import Korpora
import pandas as pd
from pprint import pprint
# from konlpy.tag import Mecab
from nltk.tokenize import word_tokenize as en_tokenizer
import sentencepiece as spm
import urllib.request
import csv
import numpy as np
from einops import rearrange, reduce, repeat
from torch.cuda import amp
from tqdm import tqdm
import wandb
import time
import copy
from collections import defaultdict
from sklearn.metrics import mean_squared_error
import joblib
import gc
import os

In [7]:
VOCAB_SIZE = 32000 + 7
SEQ_LEN = 100
PAD_IDX = 0
# Trainig Set 모집단의 크기
TRAINSET_SIZE = 120000
# 실제로 사용할 Training Set의 크기. 이 수만큼 전체 Training Set에서 Random Sampling
TRAIN_LEN = 100000
VALID_LEN = 10000
BATCH_SIZE = 2
WANDB_SAVED_PATH = ''
if 'device' not in globals():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using {device}')

Using cpu


In [8]:
import wandb
import os
# if want to run in offline mode

os.environ["WANDB_API_KEY"] = "d60a4af56f6cd9cccec7d9da1dbced7960b61310"
os.environ["WANDB_MODE"] = "dryrun"
wandb.init(project="Transformer", entity="jiwon7258")
RUN_PATH = ''


# wandb.login(key="d60a4af56f6cd9cccec7d9da1dbced7960b61310")
# os.environ["WANDB_MODE"] = "online"
# wandb.init(project="Transformer", entity="jiwon7258")
# wandb.run.name = "kaggleGpu"

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


# Create Data

In [7]:
# dataset = open_subtitles_dataset()
corpus = Korpora.load("open_subtitles", root_dir='./')



    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : TRAC (https://trac.edgewall.org/)
    Repository : http://opus.nlpl.eu/OpenSubtitles-v2018.php
    References :
        - P. Lison and J. Tiedemann, 2016, OpenSubtitles2016: Extracting Large Parallel Corpora
          from Movie and TV Subtitles. In Proceedings of the 10th International Conference on
          Language Resources and Evaluation (LREC 2016)

    This is a new collection of translated movie subtitles from http://www.opensubtitles.org/.

    [[ IMPORTANT ]]
    If you use the OpenSubtitle corpus: Please, add a link to http://www.opensubtitles.org/
    to your website and to your reports and publications produced with the data!
    I promised this when I got the data from the providers of that website!

 

[open_subtitles] download en-ko.tmx.gz: 48.1MB [00:04, 10.2MB/s]                            


decompress /content/open_subtitles/en-ko.tmx.gz


In [8]:
data = pd.DataFrame([corpus.train.pairs, corpus.train.texts], index = ['src', 'trg'])
data = data.transpose()
data.to_csv('data.txt', index=False)
data.head()

,src,trg
0,"Through the snow and sleet and hail, through t...","폭설이 내리고 우박, 진눈깨비가 퍼부어도 눈보라가 몰아쳐도 강풍이 불고 비바람이 휘..."
1,"ever faithful, ever true, nothing stops him, h...",우리의 한결같은 심부름꾼 황새 아저씨 가는 길을 그 누가 막으랴!
2,Look out for Mr Stork That persevering chap,황새 아저씨를 기다리세요
3,He'll come along and drop a bundle in your lap,찾아와 선물을 주실 거예요
4,You may be poor or rich It doesn't matter which,가난하든 부자이든 상관이 없답니다


In [9]:
data = pd.read_csv('data.txt')
data.head()

,src,trg
0,"Through the snow and sleet and hail, through t...","폭설이 내리고 우박, 진눈깨비가 퍼부어도 눈보라가 몰아쳐도 강풍이 불고 비바람이 휘..."
1,"ever faithful, ever true, nothing stops him, h...",우리의 한결같은 심부름꾼 황새 아저씨 가는 길을 그 누가 막으랴!
2,Look out for Mr Stork That persevering chap,황새 아저씨를 기다리세요
3,He'll come along and drop a bundle in your lap,찾아와 선물을 주실 거예요
4,You may be poor or rich It doesn't matter which,가난하든 부자이든 상관이 없답니다


## Sentencepiece Train

model / vocab 파일을 생성합니다

In [10]:
with open('src.txt', mode = 'w', encoding='utf8') as f:
    f.write('\n'.join(data['src']))
with open('trg.txt', mode= 'w', encoding='utf8') as f:
    f.write('\n'.join(data['trg']))

In [11]:
corpus = "src.txt"
prefix = "src"
vocab_size = VOCAB_SIZE - 7
spm.SentencePieceTrainer.train(
    f"--input={corpus} --model_prefix={prefix} --vocab_size={vocab_size + 7}" +
    " --model_type=bpe" +
    " --max_sentence_length=999999" +  # 문장 최대 길이
    " --pad_id=0 --pad_piece=[PAD]" +  # pad (0)
    " --unk_id=1 --unk_piece=[UNK]" +  # unknown (1)
    " --bos_id=2 --bos_piece=[BOS]" +  # begin of sequence (2)
    " --eos_id=3 --eos_piece=[EOS]" +  # end of sequence (3)
    " --user_defined_symbols=[SEP],[CLS],[MASK]")  # 사용자 정의 토큰


In [13]:
corpus = "trg.txt"
prefix = "trg"
vocab_size = VOCAB_SIZE - 7
spm.SentencePieceTrainer.train(
    f"--input={corpus} --model_prefix={prefix} --vocab_size={vocab_size + 7}" +
    " --model_type=bpe" +
    " --max_sentence_length=999999" +  # 문장 최대 길이
    " --pad_id=0 --pad_piece=[PAD]" +  # pad (0)
    " --unk_id=1 --unk_piece=[UNK]" +  # unknown (1)
    " --bos_id=2 --bos_piece=[BOS]" +  # begin of sequence (2)
    " --eos_id=3 --eos_piece=[EOS]" +  # end of sequence (3)
    " --user_defined_symbols=[SEP],[CLS],[MASK]")  # 사용자 정의 토큰


## SRC Data (EN) Preprocessing

In [ ]:
wandb.restore('src.model', run_path = WANDB_SAVED_PATH, root='./')
wandb.restore('src.vocab', run_path = WANDB_SAVED_PATH, root='./')
wandb.restore('trg.model', run_path = WANDB_SAVED_PATH, root='./')
wandb.restore('trg.vocab', run_path = WANDB_SAVED_PATH, root='./')

In [ ]:
sp_src = spm.SentencePieceProcessor()
sp_src.Load('src.model')
lines = [
    "I didn't at all think of it this way.",
    "I have waited a long time for someone to film",
    "[PAD] [CLS] [BOS] [EOS] [SEP] [UNK] "
]
for line in lines:
    print(sp_src.EncodeAsPieces(line))
    print(sp_src.EncodeAsIds(line))


In [15]:
def en_encode(tmpstr:str) -> np.array :
    tmpstr = np.array(sp_src.EncodeAsIds(tmpstr))

    # SEQ_LEN보다 길면 짜른다 
    if len(tmpstr) > SEQ_LEN :
        tmpstr = tmpstr[:SEQ_LEN]

    # SEQ_LEN보다 작으면 padding
    else :
        tmpstr = np.pad(tmpstr, (0, SEQ_LEN - len(tmpstr)), 'constant', constant_values = sp_src.pad_id())
    
    return tmpstr

In [16]:
# src_data는 data['src']를 참조한다. (동일 id)
src_data = data['src']

src_list = []

for idx in range(len(src_data)):
    src_list.append(en_encode(src_data[idx]))

src_list[0]

array([ 6823,    20,  4819,    80,  3222,    51,    80, 16146, 31952,
         649,    20, 31760,   301, 31952,   649,    20,  3867,    56,
       31952,   649,    20,  1475,    80,   649,    20,  3414, 31952,
         372,  3085, 31952,   372,  7886, 31952,   649,    20, 25790,
        6930,  6787, 31952,    80,    20, 10726, 10477,  3997, 31952,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0])

## TRG Data (KO) Preprocessing

In [ ]:
sp_trg = spm.SentencePieceProcessor()
sp_trg.Load('trg.model')
lines = [
    "알잘딱깔센 임마 그거 몰라?.",
    "하.. 존나 싫다...",
    "가족이 사람을 죽여서 면목이 없다거나 같이 살던 사람들이 살해당해서 책임을 느낀다거나 자신의 이런 저런 일로",
    "[PAD] [CLS] [BOS] [EOS] [SEP] [UNK] "
]
for line in lines:
    print(sp_trg.EncodeAsPieces(line))
    print(sp_trg.EncodeAsIds(line))


In [17]:
def ko_encode(tmpstr: str) -> np.array:
    tmpstr = np.array(sp_trg.EncodeAsIds(tmpstr))
    tmpstr = np.insert(tmpstr, 0, sp_trg.bos_id())

    if len(tmpstr) >= SEQ_LEN:
        # SEQ_LEN -1의 길이로 자른다
        tmpstr = tmpstr[:SEQ_LEN-1]
        # 마지막에 <eos> 토큰을 넣어줌으로써, 길이를 SEQ_LEN으로 맞춘다
        tmpstr = np.pad(tmpstr, (0, 1),
                        'constant', constant_values=sp_trg.eos_id())


    else:
        tmpstr = np.pad(tmpstr, (0, 1),
                        'constant', constant_values=sp_trg.eos_id())
        tmpstr = np.pad(tmpstr, (0, SEQ_LEN - len(tmpstr)),
                        'constant', constant_values=sp_trg.pad_id())

    return tmpstr


In [18]:
# trg_data는 data['trg']를 참조한다. (동일 id)
trg_data = data['trg']

trg_list = []

for idx in range(len(trg_data)):
    trg_list.append(ko_encode(trg_data[idx]))   

trg_list[0]


array([    2,   721, 30905, 30546, 11101,    24, 30900, 30558,   130,
       30970, 31043,  2579, 28212,  1207,   490, 30600,  2024,  5312,
       29426,   548, 15377, 23537,   168, 11300, 30546,  3611, 30841,
       30551, 29426,     3,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0])

In [ ]:
names = [
         'src.model',
         'src.vocab',
         'trg.model',
         'trg.vocab',
]
for name in names:
    wandb.save(f'{name}')

## Train / Valid Split

In [19]:
train_mask = np.random.choice(len(src_list[:TRAINSET_SIZE]), size = TRAIN_LEN, replace = False)
valid_mask = np.random.choice(len(trg_list[TRAINSET_SIZE:]), size = VALID_LEN, replace = False)

src_train = np.take(src_list, train_mask, axis = 0)
trg_train = np.take(trg_list, train_mask, axis = 0)

src_valid = np.take(src_list, valid_mask, axis = 0)
trg_valid = np.take(trg_list, valid_mask, axis = 0)



In [20]:
print(src_train.shape)
print(trg_train.shape)
print(src_valid.shape)
print(trg_valid.shape)

(100000, 100)
(100000, 100)
(10000, 100)
(10000, 100)


In [ ]:
import gc; gc.collect()

15

In [21]:
# Save data to local
import joblib

joblib.dump(src_train, 'src_train.pkl')
joblib.dump(trg_train, 'trg_train.pkl')
joblib.dump(src_valid, 'src_valid.pkl')
joblib.dump(trg_valid, 'trg_valid.pkl')
joblib.dump(src_list, 'src_list.pkl')
joblib.dump(trg_list, 'trg_list.pkl')

['trg_list.pkl']

In [23]:
# save data in wandb
names = [
         'src_train',
         'src_valid',
         'trg_train',
         'trg_valid',
         'src_list',
         'trg_list',]

for name in names:
    wandb.save(f'{name}.pkl')
    

# Load Data

In [19]:
# wandb.restore('trg_list.pkl', run_path = 'jiwon7258/Transformer/1hdb9bta', root='./')
# wandb.restore('src_list.pkl', run_path = 'jiwon7258/Transformer/1hdb9bta', root='./')
wandb.restore('src_train.pkl', run_path = WANDB_SAVED_PATH, root='./')
wandb.restore('src_valid.pkl', run_path = WANDB_SAVED_PATH, root='./')
wandb.restore('trg_train.pkl', run_path = WANDB_SAVED_PATH, root='./')
wandb.restore('trg_valid.pkl', run_path = WANDB_SAVED_PATH, root='./')

<_io.TextIOWrapper name='./trg_valid.pkl' mode='r' encoding='UTF-8'>

In [10]:
# # Load Whole Data

# src_list_path = 'src_list.pkl'
# trg_list_path = 'trg_list.pkl'
# src_list = joblib.load(src_list_path)
# trg_list = joblib.load(trg_list_path)

In [9]:
# or Train / Valid Data
src_train_path = 'src_train.pkl'
src_valid_path = 'src_valid.pkl'
trg_train_path = 'trg_train.pkl'
trg_valid_path = 'trg_valid.pkl'

src_train = joblib.load(src_train_path)
src_valid = joblib.load(src_valid_path)
trg_train = joblib.load(trg_train_path)
trg_valid = joblib.load(trg_valid_path)

# Create Dataset

In [10]:
class TrainDataset(Dataset):
    def __init__(self, src_data, trg_data):
        super().__init__()

        assert len(src_data) == len(trg_data)

        self.src_data = src_data
        self.trg_data = trg_data

    def __len__(self):
        return len(self.src_data)
        
    def __getitem__ (self, idx):
        src = self.src_data[idx]
        trg_input = self.trg_data[idx]
        trg_output = trg_input[1:SEQ_LEN]
        trg_output = np.pad(trg_output, (0,1), 'constant', constant_values =0)
        # (seq_len,)
        return src, trg_input, trg_output

train_dataset = TrainDataset(src_train, trg_train)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle= True, pin_memory=True)

In [11]:
class ValidDataset(Dataset):
    def __init__(self, src_data, trg_data):
        super().__init__()

        assert len(src_data) == len(trg_data)

        self.src_data = src_data
        self.trg_data = trg_data

    def __len__(self):
        return len(self.src_data)
        
    def __getitem__ (self, idx):
        src = self.src_data[idx]
        trg_input = self.trg_data[idx]
        trg_output = trg_input[1:SEQ_LEN]
        trg_output = np.pad(trg_output, (0,1), 'constant',constant_values= 0)

        return src, trg_input, trg_output

valid_dataset = ValidDataset(src_valid, trg_valid)
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle= False, pin_memory=True)

In [12]:
for src, trg_input, trg_output in train_dataloader:
    print(src.shape)
    print(trg_input.shape)
    print(trg_output.shape)
    break

torch.Size([2, 100])
torch.Size([2, 100])
torch.Size([2, 100])


# Transformer

## Mask Function

In [13]:
'''
Mask 행렬을 반환하는 Mask Function

Input
- Tensor
    shape (bs, seq_len)

Args
- Option
    If option is 'padding', function returns padding mask
    If option is 'lookahead', function returns lookahead mask

Output
- Tensor
    shpae (bs, seq_len, seq_len)

'''


def makeMask(tensor, option: str) -> torch.Tensor:

    '''
    tensor (bs, seq_len)
    '''
    if option == 'padding':
        tmp = torch.full_like(tensor, fill_value=PAD_IDX).to(device)
        # tmp : (bs,seq_len)
        mask = (tensor != tmp).float()
        # mask : (bs, seq_len)
        mask = repeat(mask, 'bs seq_len -> bs new_axis seq_len ',
                      new_axis=mask.shape[1])
        # mask(bs,seq_len,seq_len)

        '''
        Example of mask
        tensor([[
         [1., 1., 1., 1., 0., 0., 0., 0.],
         [1., 1., 1., 1., 0., 0., 0., 0.],
         [1., 1., 1., 1., 0., 0., 0., 0.],
         [1., 1., 1., 1., 0., 0., 0., 0.],
         [1., 1., 1., 1., 0., 0., 0., 0.],
         [1., 1., 1., 1., 0., 0., 0., 0.],
         [1., 1., 1., 1., 0., 0., 0., 0.],
         [1., 1., 1., 1., 0., 0., 0., 0.]]])
        '''

    elif option == 'lookahead':
        padding_mask = makeMask(tensor, 'padding')

        mask = torch.ones_like(padding_mask)
        mask = torch.tril(mask)
        '''
        Example of 'mask'
        tensor([[
        [1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]]])
        '''

        mask = mask * padding_mask

        '''
        Example
        tensor([[
         [1., 0., 0., 0., 0., 0., 0., 0.],
         [1., 1., 0., 0., 0., 0., 0., 0.],
         [1., 1., 1., 0., 0., 0., 0., 0.],
         [1., 1., 1., 1., 0., 0., 0., 0.],
         [1., 1., 1., 1., 0., 0., 0., 0.],
         [1., 1., 1., 1., 0., 0., 0., 0.],
         [1., 1., 1., 1., 0., 0., 0., 0.],
         [1., 1., 1., 1., 0., 0., 0., 0.]]])
        '''


    return mask


## Positional Embedding

In [14]:
# def pos_embed(input):
    # input : (bs, seq_len, hidden_dim)

    

## Multihead Attention

In [15]:
class Multiheadattention(nn.Module):
    def __init__(self, hidden_dim: int, num_head: int):
        super().__init__()

        # embedding_dim, d_model, 512 in paper
        self.hidden_dim = hidden_dim
        # 8 in paper
        self.num_head = num_head
        # head_dim, d_key, d_query, d_value, 64 in paper (= 512 / 8)
        self.head_dim = hidden_dim // num_head
        self.scale = torch.sqrt(torch.FloatTensor()).to(device)

        self.fcQ = nn.Linear(hidden_dim, hidden_dim)
        self.fcK = nn.Linear(hidden_dim, hidden_dim)
        self.fcV = nn.Linear(hidden_dim, hidden_dim)
        self.fcOut = nn.Linear(hidden_dim, hidden_dim)



    def forward(self, srcQ, srcK, srcV, mask=None):

        ##### SCALED DOT PRODUCT ATTENTION ######

        # input : (bs, seq_len, hidden_dim)
        Q = self.fcQ(srcQ)
        K = self.fcK(srcK)
        V = self.fcV(srcV)

        Q = rearrange(
            Q, 'bs seq_len (num_head head_dim) -> bs num_head seq_len head_dim', num_head=self.num_head)
        K_T = rearrange(
            K, 'bs seq_len (num_head head_dim) -> bs num_head head_dim seq_len', num_head=self.num_head)
        V = rearrange(
            V, 'bs seq_len (num_head head_dim) -> bs num_head seq_len head_dim', num_head=self.num_head)

        attention_energy = torch.matmul(Q, K_T)
        # attention_energy : (bs, num_head, seq_len, seq_len)

        if mask is not None :
            attention_energy : torch.masked_fill(attention_energy, (mask==0), -1e10)

        attention_energy = torch.softmax(attention_energy, dim = -1)
        # print(attention_energy[0,0,0,:])

        result = torch.matmul(attention_energy,V)
        # result (bs, num_head, seq_len, head_dim)

        ##### END OF SCALED DOT PRODUCT ATTENTION ######

        # CONCAT
        result = rearrange(result, 'bs num_head seq_len head_dim -> bs seq_len (num_head head_dim)')
        # result : (bs, seq_len, hidden_dim)

        # LINEAR

        result = self.fcOut(result)

        return result
        


In [16]:
# # TEST CODE #
# bs = 32
# seq_len = 200
# hidden_dim = 128
# test_tensor = torch.randn((bs,seq_len,hidden_dim))
# print(test_tensor.shape)
# test_layer = Multiheadattention(hidden_dim=hidden_dim, num_head =2)
# print(test_layer(srcQ = test_tensor, srcK = test_tensor, srcV = test_tensor).shape)

## Poistionwise Feedforward Network

In [17]:
class FFN(nn.Module):
    def __init__ (self, hidden_dim, inner_dim):
        super().__init__()

        # 512 in paper 
        self.hidden_dim = hidden_dim
        # 2048 in paper
        self.inner_dim = inner_dim 

        self.fc1 = nn.Linear(hidden_dim, inner_dim)
        self.fc2 = nn.Linear(inner_dim, hidden_dim)
        self.relu = nn.ReLU(inplace=False)

        
    def forward(self, input):
        output = input
        output = self.fc1(output)
        output2 = self.relu(output)
        output3 = self.fc2(output2)

        return output3


## Encoder Layer

In [18]:
class EncoderLayer(nn.Module):
    def __init__(self, hidden_dim, num_head, inner_dim):
        super().__init__()

        self.hidden_dim = hidden_dim
        self.num_head = num_head
        self.inner_dim = inner_dim
        
        self.multiheadattention = Multiheadattention(hidden_dim, num_head)
        self.ffn = FFN(hidden_dim, inner_dim)
        self.layerNorm = nn.LayerNorm(hidden_dim)

        self.dropout1 = nn.Dropout(p=0.1)
        self.dropout2 = nn.Dropout(p=0.1)


    def forward(self, input, mask = None):

        # input : (bs, seq_len, hidden_dim)
        
        # encoder attention
        # uses only padding mask
        output = self.multiheadattention(srcQ= input, srcK = input, srcV = input, mask = mask)
        output = self.dropout1(output)
        output = input + output
        output = self.layerNorm(output)

        output_ = self.ffn(output)
        output_ = self.dropout2(output_)
        output = output + output_
        output = self.layerNorm(output)

        # output : (bs, seq_len, hidden_dim)
        return output



## Encoder Architecture

In [19]:
class Encoder(nn.Module):
    def __init__ (self, N, hidden_dim, num_head, inner_dim):
        super().__init__()

        # N : number of encoder layer repeated 
        self.N = N
        self.hidden_dim = hidden_dim
        self.num_head = num_head
        self.inner_dim = inner_dim

        self.embedding = nn.Embedding(num_embeddings=VOCAB_SIZE, embedding_dim=hidden_dim, padding_idx=0)
        self.enc_layers = nn.ModuleList(EncoderLayer(hidden_dim, num_head, inner_dim) for _ in range(N))

        self.dropout = nn.Dropout(p=0.1)



    def forward(self, input):

        # input : (bs, seq_len)
        print(f'input shape = {input.shape}')
        mask = makeMask(input, option='padding')

        # embedding layer
        output = self.embedding(input)
        # output : (bs, seq_len, hidden_dim)

        # Positional Embedding
        # output = pos_embed(output)

        # Dropout
        output = self.dropout(output)

        # N encoder layer
        for layer in self.enc_layers:
            output = layer(output, mask)

        # output : (bs, seq_len, hidden_dim)

        return output


## Decoder Layer

In [20]:
class DecoderLayer(nn.Module):
    def __init__(self, hidden_dim, num_head, inner_dim):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.num_head = num_head
        self.inner_dim = inner_dim

        self.multiheadattention1 = Multiheadattention(hidden_dim, num_head)
        self.layerNorm1 = nn.LayerNorm(hidden_dim)
        self.multiheadattention2 = Multiheadattention(hidden_dim, num_head)
        self.layerNorm2 = nn.LayerNorm(hidden_dim)
        self.ffn = FFN(hidden_dim, inner_dim)
        self.layerNorm3 = nn.LayerNorm(hidden_dim)

        self.dropout1 = nn.Dropout(p=0.1)
        self.dropout2 = nn.Dropout(p=0.1)
        self.dropout3 = nn.Dropout(p=0.1)

    
    def forward(self, input, enc_output, paddingMask, lookaheadMask):
        # input : (bs, seq_len, hidden_dim)

        # first multiheadattention
        output = self.multiheadattention1(input, input, input, lookaheadMask)
        output = self.dropout1(output)
        output = output + input
        output = self.layerNorm1(output)

        # second multiheadattention
        output_ = self.multiheadattention2(output, enc_output, enc_output, paddingMask)
        output_ = self.dropout2(output_)
        output = output_ + output
        output = self.layerNorm2(output)

        # Feedforward Network
        output_ = self.ffn(output)
        output_ = self.dropout3(output_)
        output = output + output_
        output = self.layerNorm3(output)

        return output


## Decoder Architecture

In [21]:
class Decoder(nn.Module):
    def __init__ (self, N, hidden_dim, num_head, inner_dim):
        super().__init__()

        # N : number of encoder layer repeated 
        self.N = N
        self.hidden_dim = hidden_dim
        self.num_head = num_head
        self.inner_dim = inner_dim

        self.embedding = nn.Embedding(num_embeddings=VOCAB_SIZE, embedding_dim=hidden_dim, padding_idx=0)

        self.dec_layers = nn.ModuleList(DecoderLayer(hidden_dim, num_head, inner_dim) for _ in range(N))

        self.dropout = nn.Dropout(p=0.1)
        
        self.finalFc = nn.Linear(hidden_dim, VOCAB_SIZE)


    def forward(self, input, enc_src, enc_output):

        # input : (bs, seq_len)
        # enc_src : (bs, seq_len)
        # enc_output : (bs, seq_len,hidden_dim)

        lookaheadMask = makeMask(input, option='lookahead')
        paddingMask = makeMask(enc_src, option = 'padding')

        # embedding layer
        output = self.embedding(input)

        # Positional Embedding
        # output = pos_embed(output)

        # Dropout
        output = self.dropout(output)

        # N decoder layer
        for layer in self.dec_layers:
            output = layer(output, enc_output, paddingMask, lookaheadMask)
        # output : (bs, seq_len, hidden_dim)

        logits = self.finalFc(output)
        # logits : (bs, seq_len, VOCAB_SIZE)
        output = torch.softmax(logits, dim = -1)

        output = torch.argmax(output, dim = -1)
        # output : (bs, seq_len)



        return output, logits


## Transformer Model

In [34]:
class Transformer(nn.Module):
    def __init__(self, N = 6, hidden_dim = 512, num_head = 8, inner_dim = 2048):
        super().__init__()
        self.encoder = Encoder(N, hidden_dim, num_head, inner_dim)
        self.decoder = Decoder(N, hidden_dim, num_head, inner_dim)

    def forward(self, enc_src, dec_src):
        # enc_src : (bs, seq_len)
        # dec_src : (bs, seq_len)

        # print(f'enc_src : {enc_src.shape}')
        # print(f'dec_src : {dec_src.shape}')

        enc_output = self.encoder(enc_src)
        output, logits = self.decoder(dec_src, enc_src, enc_output.detach())
        # logits = (bs, seq_len, VOCAB_SIZE) 

        return output, logits


# Model Train

# Create Model

In [35]:
model = Transformer().to(device)

# Check Model Structure

In [36]:
from torchsummary import summary
test1 = torch.randint(low = 0, high = 1000, size = (SEQ_LEN,))
test2 = torch.randint(low = 0, high = 1000, size = (SEQ_LEN,))
summary(model, [(SEQ_LEN,), (SEQ_LEN,)], dtypes = [torch.int, torch.int])

input shape = torch.Size([2, 100])
Layer (type:depth-idx)                        Output Shape              Param #
├─Encoder: 1-1                                [-1, 100, 512]            --
|    └─Embedding: 2-1                         [-1, 100, 512]            16,387,584
|    └─Dropout: 2-2                           [-1, 100, 512]            --
|    └─ModuleList: 2                          []                        --
|    |    └─EncoderLayer: 3-1                 [-1, 100, 512]            3,151,360
|    |    └─EncoderLayer: 3-2                 [-1, 100, 512]            3,151,360
|    |    └─EncoderLayer: 3-3                 [-1, 100, 512]            3,151,360
|    |    └─EncoderLayer: 3-4                 [-1, 100, 512]            3,151,360
|    |    └─EncoderLayer: 3-5                 [-1, 100, 512]            3,151,360
|    |    └─EncoderLayer: 3-6                 [-1, 100, 512]            3,151,360
├─Decoder: 1-2                                [-1, 100]                 --
|    └─Emb

Layer (type:depth-idx)                        Output Shape              Param #
├─Encoder: 1-1                                [-1, 100, 512]            --
|    └─Embedding: 2-1                         [-1, 100, 512]            16,387,584
|    └─Dropout: 2-2                           [-1, 100, 512]            --
|    └─ModuleList: 2                          []                        --
|    |    └─EncoderLayer: 3-1                 [-1, 100, 512]            3,151,360
|    |    └─EncoderLayer: 3-2                 [-1, 100, 512]            3,151,360
|    |    └─EncoderLayer: 3-3                 [-1, 100, 512]            3,151,360
|    |    └─EncoderLayer: 3-4                 [-1, 100, 512]            3,151,360
|    |    └─EncoderLayer: 3-5                 [-1, 100, 512]            3,151,360
|    |    └─EncoderLayer: 3-6                 [-1, 100, 512]            3,151,360
├─Decoder: 1-2                                [-1, 100]                 --
|    └─Embedding: 2-3                        

# Weight Init

In [ ]:
for param in model.named_parameters():
    if 'weight' in param[0] and 'layerNorm' not in param[0] :
        torch.nn.init.xavier_uniform_(param[1])
        

# Optimizer

In [ ]:
optimizer = torch.optim.Adam(params = model.parameters())

# Loss Function

In [ ]:
def criterion(logits: torch.tensor, targets: torch.tensor):
    return nn.CrossEntropyLoss(ignore_index=PAD_IDX)(logits.view(-1,VOCAB_SIZE), targets.view(-1))

# Training Function

In [ ]:
def train_one_epoch(model, optimizer, scheduler, dataloader, device, epoch):
    # train 모드로 변경
    model.train()

    # for the Mixed Precision
    # Pytorch 예제 : https://pytorch.org/docs/stable/notes/amp_examples.html#amp-examples
    scaler = amp.GradScaler()

    dataset_size = 0
    running_loss = 0

    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    
    for step, (src, trg_input, trg_output) in bar:
        src = src.to(device)
        trg_input = trg_input.to(device)
        trg_output = trg_output.to(device)

        batch_size = src.shape[0]

        with amp.autocast(enabled=True):
            output, logits = model(enc_src = src, dec_src = trg_input)
            loss = criterion(logits, trg_output)
            
        # logits (bs, seq_len, VOCAB_SIZE)
        # trg_output (bs, seq_len)

        # loss를 Scale
        # Scaled Grdients를 계산(call)하기 위해 scaled loss를 backward()
        scaler.scale(loss).backward()
        # loss.backward()

        # scaler.step() first unscales the gradients of the optimizer's assigned params.
        # If these gradients do not contain infs or NaNs, optimizer.step() is then called,
        # otherwise, optimizer.step() is skipped.
        scaler.step(optimizer)
        # optimizer.step()
        
        # Updates the scale for next iteration.
        scaler.update()

        # zero the parameter gradients
        optimizer.zero_grad()

        # change learning rate by Scheduler
        if scheduler is not None:
            scheduler.step()

        # loss.item()은 loss를 Python Float으로 반환
        # loss.item()은 batch data의 average loss이므로, sum of loss를 구하기 위해 batch_size를 곱해준다
        running_loss += loss.item() * batch_size
        dataset_size += batch_size

        epoch_loss = running_loss / dataset_size

        bar.set_postfix(
            Epoch=epoch, Train_Loss=epoch_loss, LR=optimizer.param_groups[0]["lr"]
        )

    # Garbage Collector
    gc.collect()

    return epoch_loss

# Validation Function

In [ ]:
@torch.no_grad()
def valid_one_epoch(model, dataloader, device, epoch):
    model.eval()

    dataset_size = 0
    running_loss = 0


    bar = tqdm(enumerate(dataloader), total=len(dataloader))

    for step, (src, trg_input, trg_output) in bar:
        src = src.to(device)
        trg_input = trg_input.to(device)
        trg_output = trg_output.to(device)

        batch_size = src.shape[0]

        output, logits = model(enc_src = src, dec_src = trg_input)
        loss = criterion(logits, trg_output)

        running_loss += loss.item() * batch_size
        dataset_size += batch_size

        # 실시간으로 정보를 표시하기 위한 epoch loss
        val_loss = running_loss / dataset_size

        bar.set_postfix(
            Epoch=epoch, Valid_Loss=val_loss, LR=optimizer.param_groups[0]["lr"]
        )

    gc.collect()

    return val_loss

In [ ]:
def run_training(
    model,
    optimizer,
    scheduler,
    device,
    num_epochs,
    metric_prefix="",
    file_prefix="",
    early_stopping=True,
    early_stopping_step=10,
):
    # To automatically log graidents
    wandb.watch(model, log_freq=100)

    if torch.cuda.is_available():
        print("[INFO] Using GPU:{}\n".format(torch.cuda.get_device_name()))

    start = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = np.inf
    history = defaultdict(list)
    early_stop_counter = 0

    # num_epochs만큼, train과 val을 실행한다
    for epoch in range(1, num_epochs + 1):
        gc.collect()

        train_epoch_loss = train_one_epoch(
            model,
            optimizer,
            scheduler,
            dataloader= train_dataloader,
            device=device,
            epoch=epoch,
        )

        val_loss = valid_one_epoch(
            model, valid_dataloader, device=device, epoch=epoch
        )

        history[f"{metric_prefix}Train Loss"].append(train_epoch_loss)
        history[f"{metric_prefix}Valid Loss"].append(val_loss)

        # Log the metrics
        wandb.log(
            {
                f"{metric_prefix}Train Loss": train_epoch_loss,
                f"{metric_prefix}Valid Loss": val_loss,
            }
        )

        print(f"Valid Loss : {val_loss}")

        # deep copy the model
        if val_loss <= best_loss:
            early_stop_counter = 0

            print(
                f"Validation Loss improved( {best_loss} ---> {val_loss}  )"
            )

            # Update Best Loss
            best_loss = val_loss
            
            # Update Best Model Weight
            # run.summary['Best RMSE'] = best_loss
            best_model_wts = copy.deepcopy(model.state_dict())

            PATH = "{}epoch{:.0f}_Loss{:.4f}.bin".format(file_prefix, epoch, best_loss)
            torch.save(model.state_dict(), PATH)
            torch.save(model.state_dict(), f"{file_prefix}best_{epoch}epoch.bin")
            # Save a model file from the current directory
            wandb.save(PATH)

            print(f"Model Saved")

        elif early_stopping:
            early_stop_counter += 1
            if early_stop_counter > early_stopping_step:
                break

        print()

    end = time.time()
    time_elapsed = end - start
    print(
        "Training complete in {:.0f}h {:.0f}m {:.0f}s".format(
            time_elapsed // 3600,
            (time_elapsed % 3600) // 60,
            (time_elapsed % 3600) % 60,
        )
    )
    print("Best Loss: {:.4f}".format(best_loss))

    # load best model weights
    model.load_state_dict(best_model_wts)

    return model, history

In [ ]:
run_training(
    model = model,
    optimizer = optimizer,
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, T_max=100, eta_min=1e-6),
    device = device,
    num_epochs = 20,
    metric_prefix="",
    file_prefix="",
    early_stopping=True,
    early_stopping_step=10,
)

/home/jiwon/anaconda3/lib/python3.9/site-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")
  0%|          | 0/782 [00:00<?, ?it/s]/home/jiwon/anaconda3/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


# Inference

In [38]:
src_sentence = 'i don\'t love you, pleas go away from me. Do you understand?'
enc_src = en_encode(src_sentence)
last = None
dec_src = torch.tensor(SOS_TOKEN)
enc_output = model.encoder(enc_src)

while last is not EOS_TOKEN :
    




NameError: name 'en_encode' is not defined